# Detect Buyer and Seller Signatures with Tensorlake SDK

This is a quick example of how to parse a real estate purchase agreement and extract the buyer and seller signatures, names, and signature dates.

This notebook pairs with [this example](https://docs.tensorlake.ai/examples/cookbooks/signature-detection/detect-buyer-and-seller-signatures-sdk).
Learn more about signature detection with Tensorlake on [our docs](https://docs.tensorlake.ai/document-ingestion/parsing/signature).

## Prerequisites
Make sure you have a [Tensorlake API key](https://docs.tensorlake.ai/platform/authentication#api-keys)

## Step 1: Install Tensorlake and set your API key

In [ ]:
!pip install tensorlake

In [ ]:
%env TENSORLAKE_API_KEY=YOUR_TENSORLAKE_API_KEY

### Step 2: Import packages, setup client, and define file path

In [ ]:
import json
from pydantic import BaseModel

from tensorlake.documentai import (
        DocumentAI,
        ParseStatus,
        ParsingOptions,
        StructuredExtractionOptions
      )

# Initialize the client and add your API Key
doc_ai = DocumentAI()

# Upload the document
file_path = "https://tlake.link/lease-agreement"

### Step 3: Define your structured extraction schema

In [ ]:
# Create a schema
class Buyer(BaseModel):
    buyer_name: str
    buyer_signature_date: str
    buyer_signed: bool

class Seller(BaseModel):
    seller_name: str
    seller_signature_date: str
    seller_signed: bool

class RealEstateSchema(BaseModel):
    buyer: Buyer
    seller: Seller


### Step 4: Set your parsing options, and parse

In [ ]:
real_state_extraction = StructuredExtractionOptions(
    schema_name="real-estate-schema",
    json_schema=RealEstateSchema
)

# Configure parsing options
parsing_options = ParsingOptions(
    signature_detection=True
)

parse_id = doc_ai.parse(
    file=file_path,
    parsing_options=parsing_options,
    structured_extraction_options=[real_state_extraction],
)

print(f"Parse job submitted with ID: {parse_id}")

# Get the result
result = doc_ai.wait_for_completion(parse_id)

if(result.status == ParseStatus.FAILURE):
    print("Parse job failed!")
    exit(1)

print("Successully parsed the document!")

Parse job submitted with ID: parse_CNGbHfrQ7t7c7grzTGDk9
waiting 5 s…
parse status: processing
waiting 5 s…
parse status: processing
waiting 5 s…
parse status: processing
waiting 5 s…
parse status: processing
waiting 5 s…
parse status: processing
waiting 5 s…
parse status: successful
Successully parsed the document!


### Step 5: Print your results

In [ ]:
print("========Structured Data========")
print(json.dumps(result.structured_data[0].data, indent=2))

print("\n\n")
print("========Chunks========")
for chunk in result.chunks:
    print(chunk.content)

========Structured Data========
{
  "buyer": {
    "buyer_name": "Nova Ellison",
    "buyer_signature_date": "September 10, 2025",
    "buyer_signed": true
  },
  "seller": {
    "seller_name": "Juno Vega",
    "seller_signature_date": "September 10, 2025",
    "seller_signed": true
  }
}



========Chunks========

## RESIDENTIAL REAL ESTATE PURCHASE AGREEMENT

I. THE PARTIES. This Real Estate Purchase Agreement ("Agreement") made on September 20 20 25 , ("Effective Date") between:
Buyer: Nova Ellison with a mailing address of 123 Tensor Rd, San Francisco, CA 99999 ("Buyer"), who agrees to buy, and:
Seller: Juno Vega with a mailing address of 456 Lake Rd, San Francisco, CA 99999 ("Seller"), who agrees to sell and convey real and personal property as described in Sections II & III.
Buyer and Seller are each referred to herein as a "Party" and, collectively, as the "Parties."
II. LEGAL DESCRIPTION. The real property is a: (check one) [x] - Single-Family Home [ ] - Condominium [ ] - Plann